In [ ]:
!pip install -q transformers sentencepiece
!pip install -q pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 18.7 MB/s eta 0:00:00


In [ ]:
from transformers import VisionEncoderDecoderConfig

image_size = [1280, 960]
max_length = 768

# update image_size of the encoder
# during pre-training, a larger image size was used
config = VisionEncoderDecoderConfig.from_pretrained("rhovhannisyan/dmr-invoice-extractor")
# config.encoder.image_size = image_size # (height, width)
# update max_length of the decoder (for generation)
# config.decoder.max_length = max_length

In [ ]:
config.add_cross_attention =True

In [ ]:
config.decoder.output_attentions=True
config.encoder.output_attentions=True
id2labels = {0:'payment_terms',1:'date',2:'currency',3:'country',4:'amount',5:'Rname',6:'Bname'}
labels2id = {v:k for k, v in id2labels.items()}
config.id2label = id2labels
config.label2id = labels2id
config.decoder.cross_attention_hidden_size=1024

In [ ]:
from transformers import DonutProcessor, VisionEncoderDecoderModel,AutoProcessor


processor = DonutProcessor.from_pretrained("rhovhannisyan/dmr-invoice-extractor")
model = VisionEncoderDecoderModel.from_pretrained("rhovhannisyan/dmr-invoice-extractor", config=config)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
processor2 = AutoProcessor.from_pretrained("bert-base-cased")
processor2.convert_tokens_to_ids('amount')

2971

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = os.listdir("/content/drive/MyDrive/Invoies_final_Tapas_Toushima")
y = data_path[-1]
data_path.remove(y)

In [ ]:
d = []
d.append('/content/drive/MyDrive/Invoies_final_Tapas_Toushima/0_Others(INVOICE 1036_JPY).pdf.json')
d.append('/content/drive/MyDrive/Invoies_final_Tapas_Toushima/0_Others(ANglia_CPCM805712339254CRE001_1).pdf.json')
d.append('/content/drive/MyDrive/Invoies_final_Tapas_Toushima/0_Others(INV 877_CPCM401322340931CRE001_2).pdf.json')
d.append('/content/drive/MyDrive/Invoies_final_Tapas_Toushima/0_Others(PROFORMA INVOICE_CPCM603632338957CRE001_1).pdf.json')

In [ ]:
train_data_path = ['/content/drive/MyDrive/Invoies_final_Tapas_Toushima/' + path for path in data_path if path[-3:] == 'jpg']
label_data_path = [path.replace('jpg','json')  for path in train_data_path]

In [ ]:
df_path = pd.DataFrame(columns = ['data_path', 'label_path'])
df_path.data_path = train_data_path
df_path.label_path = label_data_path

In [ ]:
df_path = df_path[~df_path['label_path'].isin(d)]

In [ ]:
import json
import random
from typing import Any, List, Tuple

import torch
from torch.utils.data import Dataset

added_tokens = []

class DonutDataset(Dataset):
    """
    PyTorch Dataset for Donut. This class takes a HuggingFace Dataset as input.

    Each row, consists of image path(png/jpg/jpeg) and gt data (json/jsonl/txt),
    and it will be converted into pixel_values (vectorized image) and labels (input_ids of the tokenized string).

    Args:
        dataset_name_or_path: name of dataset (available at huggingface.co/datasets) or the path containing image files and metadata.jsonl
        max_length: the max number of tokens for the target sequences
        split: whether to load "train", "validation" or "test" split
        ignore_id: ignore_index for torch.nn.CrossEntropyLoss
        task_start_token: the special token to be fed to the decoder to conduct the target task
        prompt_end_token: the special token at the end of the sequences
        sort_json_key: whether or not to sort the JSON keys
    """

    def __init__(
        self,
        dataset_name_or_path: str,
        max_length: int,
        ignore_id: int = -100,
        task_start_token: str = "",
        prompt_end_token: str = None,
        sort_json_key: bool = True,
        data_path = data_path
    ):
        super().__init__()
        self.max_length = max_length
        self.ignore_id = ignore_id
        self.task_start_token = task_start_token
        self.prompt_end_token = prompt_end_token if prompt_end_token else task_start_token
        self.sort_json_key = sort_json_key

        self.dataset = data_path
        self.dataset_length = data_path.shape[0]

        self.gt_token_sequences = []
        ground_truth =[]
        for file_path in self.dataset.label_path:
          with open(file_path, 'r') as file:
            json_data = json.load(file)
            ground_truth = [json_data]
            # print(ground_truth)
          self.gt_token_sequences.append([self.json2token(gt_json, sort_json_key=self.sort_json_key) + processor.tokenizer.eos_token
                  for gt_json in ground_truth  # load json from list of json
              ]
          )
            # print(len(self.gt_token_sequences))

        self.add_tokens([self.task_start_token, self.prompt_end_token])
        self.prompt_end_token_id = processor.tokenizer.convert_tokens_to_ids(self.prompt_end_token)

    def json2token(self, obj: Any, update_special_tokens_for_json_key: bool = True, sort_json_key: bool = True):
        """
        Convert an ordered JSON object into a token sequence
        """
        if type(obj) == dict:
            if len(obj) == 1 and "text_sequence" in obj:
                return obj["text_sequence"]
            else:
                output = ""
                if sort_json_key:
                    keys = sorted(obj.keys(), reverse=True)
                else:
                    keys = obj.keys()
                for k in keys:
                    if update_special_tokens_for_json_key:
                        added_tokens.append(fr"<s_{k}>") if fr"<s_{k}>" not in added_tokens else None
                        added_tokens.append(fr"</s_{k}>") if fr"</s_{k}>" not in added_tokens else None
                        self.add_tokens([fr"", fr""])
                    output += (
                        fr"<s_{k}"
                        + self.json2token(obj[k], update_special_tokens_for_json_key, sort_json_key)
                        + fr"</s_{k}"
                    )
                return output
        # elif type(obj) == list:
        #     return r"<sep/>".join(
        #         [self.json2token(item, update_special_tokens_for_json_key, sort_json_key) for item in obj]
        #     )
        else:
            obj = str(obj)
            if f"<{obj}/>" in added_tokens:
                obj = f"<{obj}/>"  # for categorical special tokens
            return obj

    def add_tokens(self, list_of_tokens: List[str]):
        """
        Add special tokens to tokenizer and resize the token embeddings of the decoder
        """
        newly_added_num = processor.tokenizer.add_tokens(list_of_tokens)
        if newly_added_num > 0:
            model.decoder.resize_token_embeddings(len(processor.tokenizer))
            added_tokens.extend(list_of_tokens)

    def __len__(self) -> int:
        return self.dataset_length

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Load image from image_path of given dataset_path and convert into input_tensor and labels
        Convert gt data into input_ids (tokenized string)
        Returns:
            input_tensor : preprocessed image
            input_ids : tokenized gt_data
            labels : masked labels (model doesn't need to predict prompt and pad token)
        """
        data= self.dataset.data_path.iloc[idx]

        img = Image.open(data)
        img = np.array(img)

        # inputs
        pixel_values = processor(img, random_padding=True, return_tensors="pt").pixel_values
        pixel_values = pixel_values.squeeze()

        # targets
        # print(len(self.gt_token_sequences))
        processor.tokenizer.add_special_tokens({"additional_special_tokens": added_tokens + [self.task_start_token] + [self.prompt_end_token]})
        target_sequence = self.gt_token_sequences[idx]  # can be more than one, e.g., DocVQA Task 1
        input_ids = processor.tokenizer(
            target_sequence,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )["input_ids"].squeeze(0)

        labels = input_ids.clone()
        labels[labels == processor.tokenizer.pad_token_id] = self.ignore_id  # model doesn't need to predict pad token
        # labels[: torch.nonzero(labels == self.prompt_end_token_id).sum() + 1] = self.ignore_id  # model doesn't need to predict prompt (for VQA)
        return pixel_values, labels, target_sequence

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_df, temp_df =train_test_split(df_path, test_size =0.3)

In [ ]:
validate_df, test_df =train_test_split(temp_df,test_size=0.3)

In [ ]:
processor2.size = image_size[::-1] # should be (width, height)
processor2.do_align_long_axis = False

train_dataset = DonutDataset("rhovhannisyan/dmr-invoice-extractor", max_length=max_length,task_start_token="", prompt_end_token="",
                             sort_json_key=True,data_path = train_df
                             )
print('train_dataset created')
val_dataset = DonutDataset("rhovhannisyan/dmr-invoice-extractor", max_length=max_length,task_start_token="", prompt_end_token="",
                             sort_json_key=True,data_path = validate_df
                             )
print('val_dataset created')

test_dataset =DonutDataset("rhovhannisyan/dmr-invoice-extractor", max_length=max_length,task_start_token="", prompt_end_token="",
                             sort_json_key=True,data_path = test_df)
print('test_dataset created')


In [ ]:
from torch.utils.data import DataLoader

# feel free to increase the batch size if you have a lot of memory
# I'm fine-tuning on Colab and given the large image size, batch size > 1 is not feasible
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)
test_dataloader = DataLoader(test_dataset,batch_size=1,shuffle=False)

In [ ]:
# Change the decoder_start_token_id to 0.
model.config.decoder_start_token_id = 0

# Change the encoder to use the add_cross_attention argument.
model.config.encoder.add_cross_attention = True

# Change the decoder to use the cross_attention_hidden_size argument.

In [ ]:
from pathlib import Path
import re
from nltk import edit_distance
import numpy as np
import math

from torch.nn.utils.rnn import pad_sequence
from torch.optim.lr_scheduler import LambdaLR

import pytorch_lightning as pl
from pytorch_lightning.utilities import rank_zero_only


class DonutModelPLModule(pl.LightningModule):
    def __init__(self, config, processor, model):
        super().__init__()
        self.config = config
        self.processor = processor
        self.model = model

    def training_step(self, batch, batch_idx):
        pixel_values, labels, _ = batch

        outputs = self.model(pixel_values, labels=labels)
        loss = outputs.loss
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx, dataset_idx=0):
        pixel_values, labels, answers = batch
        batch_size = pixel_values.shape[0]
        # we feed the prompt to the model
        decoder_input_ids = torch.full((batch_size, 1), self.model.config.decoder_start_token_id, device=self.device)

        outputs = self.model.generate(pixel_values,
                                   decoder_input_ids=decoder_input_ids,
                                   max_length=max_length,
                                   early_stopping=False,
                                   pad_token_id=self.processor.tokenizer.pad_token_id,
                                   eos_token_id=self.processor.tokenizer.eos_token_id,
                                   use_cache=True,
                                   num_beams=1,
                                   bad_words_ids=[[self.processor.tokenizer.unk_token_id]],
                                   return_dict_in_generate=True,)

        predictions = []
        for seq in self.processor.tokenizer.batch_decode(outputs.sequences):
            seq = seq.replace(self.processor.tokenizer.eos_token, "").replace(self.processor.tokenizer.pad_token, "")
            seq = re.sub(r"<.*?>", "", seq, count=1).strip()  # remove first task start token
            predictions.append(seq)

        scores = []

        pattern = r"(?<=>) "
        replacement = ""
        count = 1
        for pred, answer in zip(predictions, answers):
          pred = re.sub(r"<.*?>", "", pred, count=1).strip()  # remove first task start token
          pred = re.sub(r"(?<=>)|(?=</s_)", "", pred, count=1)
          answer = answer[0].replace(self.processor.tokenizer.eos_token, "")
          scores.append(edit_distance(pred, answer) / max(len(pred), len(answer)))

          if self.config.get("verbose", False) and len(scores) == 1:
              print(f"Prediction: {pred}")
              print(f"    Answer: {answer}")
              print(f" Normed ED: {scores[0]}")

        self.log("val_edit_distance", np.mean(scores))

        return scores

    def configure_optimizers(self):
        # you could also add a learning rate scheduler if you want
        optimizer = torch.optim.Adam(self.parameters(), lr=self.config.get("lr"))

        return optimizer

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return val_dataloader

In [ ]:
conf = {"max_epochs":20,
          "check_val_every_n_epoch":1,
          "gradient_clip_val":1.0,
          "num_training_samples_per_epoch": 32,
          "lr":2e-5,
          "train_batch_sizes": [10],
          "val_batch_sizes": [1],
          "seed":42,
          "num_nodes": 1,
          "warmup_steps": 300, # 800/8*30/10, 10%
          "result_path": "./result",
          "verbose": True,
          }

model_module = DonutModelPLModule(conf, processor, model)

In [ ]:
added_tokens

['<s_payment_terms>',
 '</s_payment_terms>',
 '<s_date>',
 '</s_date>',
 '<s_currency>',
 '</s_currency>',
 '<s_country>',
 '</s_country>',
 '<s_amount>',
 '</s_amount>',
 '<s_Rname>',
 '</s_Rname>',
 '<s_Bname>',
 '</s_Bname>']

In [ ]:
from pytorch_lightning.callbacks import Callback, EarlyStopping

trainer = pl.Trainer(
        accelerator="gpu",
        devices=1,
        max_epochs=conf.get("max_epochs"),
        val_check_interval=conf.get("val_check_interval"),
        check_val_every_n_epoch=conf.get("check_val_every_n_epoch"),
        gradient_clip_val=conf.get("gradient_clip_val"),
        precision=16, # we'll use mixed precision
        num_sanity_val_steps=0,
)

trainer.fit(model_module)

/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:565: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                      | Params
----------------------------------------------------
0 | model | VisionEncoderDecoderModel | 201 M 
----------------------------------------------------
201 M     Trainable params
0         Non-trainable para

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Prediction: <s_payment_terms100% Advance</s_payment_terms<s_dateJun 7.2023</s_date<s_currencyUSD</s_currencyUSD</s_currency<s_country['India']</s_country<s_amount<s_amount<s_amount<s_Rname<s_Rname<s_Rname<s_Rname<s_RnameKNPORTER:</s_Rname<s_Bname<s_Bname<s_Bname<s_Bname<s_Bname<s_Bname<s_Bname<s_Bname
    Answer: <s_payment_termsT/T in Advance before jun.8,2023</s_payment_terms<s_dateJun,7,2023</s_date<s_currencyUSD</s_currency<s_country['Taiwan', 'India']</s_country<s_amount506.00</s_amount<s_RnameUK Chemial Asia inc</s_Rname<s_BnameJohn Morgan Printexport</s_Bname
 Normed ED: 0.4689655172413793


...

Validation: |          | 0/? [00:00<?, ?it/s]

Prediction: <s_payment_terms100% Advance</s_payment_terms<s_dateJun.7,2023</s_date<s_currencyUSD</s_currency<s_country['INIDA', 'SING']</s_country<s_amount506.00</s_amount<s_RnameUK Chemical Asia Inc</s_Rname<s_BnameBILL TO:</s_Bname
    Answer: <s_payment_termsT/T in Advance before jun.8,2023</s_payment_terms<s_dateJun,7,2023</s_date<s_currencyUSD</s_currency<s_country['Taiwan', 'India']</s_country<s_amount506.00</s_amount<s_RnameUK Chemial Asia inc</s_Rname<s_BnameJohn Morgan Printexport</s_Bname
 Normed ED: 0.23346303501945526
Prediction: <s_payment_terms100 % ADVANCE</s_payment_terms<s_date07-06-2023</s_date<s_currencyUSD</s_currency<s_country['KOREA', 'INDIA']</s_country<s_amount10000 USD</s_amount<s_RnameAMANINTERNATIONA</s_Rname<s_BnameBHAGWATI OVERSEAS</s_Bname
    Answer: <s_payment_terms100 % ADVANCE</s_payment_terms<s_date07-06-2023</s_date<s_currencyUSD</s_currency<s_country['India', 'Korea']</s_country<s_amount10000.00</s_amount<s_RnameBHAGWATI OVERSEAS</s_Rname<s_BnameAMA

Validation: |          | 0/? [00:00<?, ?it/s]

Prediction: <s_payment_terms100% Advance before JinxJapan 2023</s_payment_terms<s_dateJun.07.2023</s_date<s_currencyUSD</s_currency<s_country['India', 'Ita']</s_country<s_amount506.00</s_amount<s_RnameXiD.Singapore</s_Rname<s_BnameBILL TO</s_Bname
    Answer: <s_payment_termsT/T in Advance before jun.8,2023</s_payment_terms<s_dateJun,7,2023</s_date<s_currencyUSD</s_currency<s_country['Taiwan', 'India']</s_country<s_amount506.00</s_amount<s_RnameUK Chemial Asia inc</s_Rname<s_BnameJohn Morgan Printexport</s_Bname
 Normed ED: 0.2529182879377432
Prediction: <s_payment_terms100% ADVANCE</s_payment_terms<s_date07-06-2023</s_date<s_currencyUSD</s_currency<s_country['KOREA', 'INDIA']</s_country<s_amount10000 USD</s_amount<s_RnameGYEONGOLDO YANGU SI ELINVIDON MYON</s_Rname<s_BnameBHAGWATI OVERSEAS</s_Bname
    Answer: <s_payment_terms100 % ADVANCE</s_payment_terms<s_date07-06-2023</s_date<s_currencyUSD</s_currency<s_country['India', 'Korea']</s_country<s_amount10000.00</s_amount<s_RnameBHAGWAT

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [ ]:
test_df.head()

,data_path,label_path
2,/content/drive/MyDrive/Invoies_final_Tapas_Tou...,/content/drive/MyDrive/Invoies_final_Tapas_Tou...
17,/content/drive/MyDrive/Invoies_final_Tapas_Tou...,/content/drive/MyDrive/Invoies_final_Tapas_Tou...
157,/content/drive/MyDrive/Invoies_final_Tapas_Tou...,/content/drive/MyDrive/Invoies_final_Tapas_Tou...
158,/content/drive/MyDrive/Invoies_final_Tapas_Tou...,/content/drive/MyDrive/Invoies_final_Tapas_Tou...
73,/content/drive/MyDrive/Invoies_final_Tapas_Tou...,/content/drive/MyDrive/Invoies_final_Tapas_Tou...


In [ ]:
import re
import json
import torch
from tqdm.auto import tqdm
import numpy as np

# from donut import JSONParseEvaluator

device = "cuda" if torch.cuda.is_available() else "cpu"
output_list =trainer.validate(model=model_module.to(device), dataloaders=test_dataloader)

# scores = {"accuracies": accs, "mean_accuracy": np.mean(accs)}
# print(scores, f"length : {len(accs)}")

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Validation: |          | 0/? [00:00<?, ?it/s]

Prediction: <s_payment_terms</s_payment_terms<s_date01 JUNE 2023</s_date<s_currencyUSD</s_currency<s_country['INDIA', 'UAE']</s_country<s_amount6156.00</s_amount<s_RnameGenera Trading Polymers Racyling Papers</s_Rname<s_BnameAMAR JYOTI INDUSTRIES</s_Bname
    Answer: <s_payment_terms</s_payment_terms<s_date01 JUNE 2023</s_date<s_currencyUSD</s_currency<s_country['India']</s_country<s_amount13856.50</s_amount<s_RnameAMAR JYOTI INDUSTRIES</s_Rname<s_BnameAMAR JYOTI INDUSTRIES</s_Bname
 Normed ED: 0.20987654320987653
Prediction: <s_payment_terms100% Advance</s_payment_terms<s_date08/06/2023</s_date<s_currencyUSD</s_currency<s_country['China', 'India']</s_country<s_amount14,200.00</s_amount<s_RnameWNM ANHIWANNA ELECTRIC MACHINE TECHNOLOGY CO.,LTD</s_Rname<s_BnameGUATT CODE</s_Bname
    Answer: <s_payment_termsT/T 100% advance before production</s_payment_terms<s_date10th March 2023</s_date<s_currencyUSD</s_currency<s_country['India', 'China']</s_country<s_amount1843.00</s_amount<s_RnameANH

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     val_edit_distance     │    0.2259802978583115     │
└───────────────────────────┴───────────────────────────┘

In [ ]:
torch.save(
    model_module.model.state_dict(),
    "/content/drive/MyDrive/donut_inf/model.pt"
)
# torch.save(model_module.mlp.state_dict(), "mlp.pt")

In [ ]:
torch.save(model_module.processor,"/content/drive/MyDrive/donut_inf/processor.pt")

In [ ]:
torch.save(config,"/content/drive/MyDrive/donut_inf/config.pt")